In [39]:
import geopandas as gpd
import pandas as pd
import requests

In [40]:
from bs4 import BeautifulSoup

## Open GIS Data

In [45]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Deaths data

In [46]:
url = "https://www.americanwarlibrary.com/vietnam/vwc14.htm"

In [47]:
r = requests.get(url)

In [48]:
soup = BeautifulSoup(r.text, "html.parser")

In [49]:
th_div = soup.find_all("table")[1].find_all("th")[4:]

In [50]:
states = [th.text for th in th_div[::3]]
total_deaths = [int(th.text) for th in th_div[1::3]]
per_100k = [float(th.text) for th in th_div[2::3]]

In [53]:
vietnam_deaths_df = pd.DataFrame(
    {"NAME": states, "DEATHS": total_deaths, "PER_100k": per_100k}
)

In [67]:
vietnam_deaths_df = vietnam_deaths_df.replace("Washg'tn", "Washington")
vietnam_deaths_df = vietnam_deaths_df.replace(
    "Dist of Columbia", "District of Columbia"
)

## Combine Data

In [68]:
vietnam_deaths_gdf = states_df.merge(vietnam_deaths_df, on="NAME", how="left")

In [69]:
vietnam_deaths_gdf = vietnam_deaths_gdf[
    ["STUSPS", "NAME", "geometry", "DEATHS", "PER_100k"]
]

In [70]:
vietnam_deaths_gdf = vietnam_deaths_gdf.to_crs(5070)

In [71]:
vietnam_deaths_gdf.to_file("data/vietnam_deaths.gpkg")